In [ ]:
import re
labels_mapping = {
        'Education':  ['Education', 'Required Education'],
        'Certification':   ['Certification','Required Certification'],
        'Qualifications':   ['Qualifications', 'Required Qualifications'],
        'WorkExperience': ['Work Experience', 'Required Work  Experience', 'work', 'Experience Level', 'Experience'],
        'TechnicalSkills':   ['Technical Skills', 'Technical'],
        'SoftSkill':    ['Required Soft Skill','Soft Skill','Required Soft','Required_Soft','Soft_Skill', 'soft'],
        'HardSkill': ['Required Hard Skill','Soft Skill','Required Hard','Required_Hard','Soft_Hard','hard','Hard'],
        'Skills': ['Skills'],
        'Proficiencies':  ['Proficiencies', 'Required Proficiencies'], 
        'Benefits': ['Benefits'],
        'CompanyCulture': ['Company Culture/Values', 'company_culture', 'Company Values', 'Company Culture', 'Company_values','Company', 'Company_values'],
        'qualifications': ['Requirement'],
        "Product" : ["product"],
        "Service" : ["service"],
        "Upcoming" : ["upcoming"],
        "JobDuties" : ["jobDuties", "duties"],
    }  
keynames = [
     ['Education','Required Education'],
    ['Certification','Required Certification'],
    ['Qualifications', 'Required Qualifications'],
    ['Work Experience', 'Required Work Experience', 'work', 'Experience Level', 'Experience'],
     ['Technical Skills', 'Technical'],
     ['Required Hard Skill','Soft Skill','Required Hard','Required_Hard','Soft_Hard','hard'],
     ['Required Soft Skill','Soft Skill','Required Soft','Required_Soft','Soft_Skill', 'soft'],
    ['Skill'],
     ['Proficiencies', 'Required Proficiencies'], 
     ['Company Culture/Values', 'Company Values', 'Company Culture', 'Company_values','Company', 'Company_values'],
     ['Benefits'], ["product"],  ["service"],  ["upcoming"],  ["jobDuties", "job", "duties"]
]  
def parse_missformed_json(text):
    parsed_data = {}
    current_key = None
    current_array = None
    for line in text.split('\n'):
        match_key = re.match(r'\s*"([^"]*)"\s*:', line)
        if match_key:
            #print(match_key)
            current_key = match_key.group(1)
            if any(current_key.lower() in map(str.lower, sublist) for sublist in keynames):
                #print(current_key)
                current_array = [] 
                matches = re.findall(r'"([^"]*)"', line)
                #print("matches",matches) 
                for value in matches[1:]:
                    if any(value.lower() in map(str.lower, sublist) for sublist in keynames):
                        continue
                    #print(value)
                    current_array.append(value)
                   
                parsed_data[current_key] = current_array
    return parsed_data
 

def get_label(key):  
    for category, keywords in labels_mapping.items(): 
     
        if any(keyword.lower() in key.lower() for keyword in keywords):
            return category 
    return ""
  
def extract_json_from_text(text):
    # Find the index of the first '{' character
    start_index = text.find('{')
    if start_index == -1:
        return None
    
    # Find the index of the last '}' character
    end_index = text.rfind('}')
    if end_index == -1:
        return None
    
    # Extract the substring between the first '{' and last '}' characters
    json_text = text[start_index:end_index+1]
    
    return json_text


In [ ]:
import json
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

jobs_posting_file_path = r'./job_postings.csv'
df = pd.read_csv(jobs_posting_file_path)
rawdata = []
filename = r"./data.json"   

with open(filename, "r") as file:
    rawdata.extend(json.load(file)) 
 
print(len(rawdata)) 

### Convert the raw text in the data.json into actual JSON objects

In [ ]:
import json

all_data = []
err = 0
for item in rawdata:
    sentence = item.get('description', item.get('text', ''))
    try:
        text = extract_json_from_text(item['output'])
        try: 
            all_data.append({"sentence":sentence, "output" : json.loads(text) })   
        except Exception as e:    
            print(e)
            text = extract_json_from_text(text)
            all_data.append({"sentence":sentence, "output" : json.loads(text) })   
            pass
    except Exception as e:   
        try:     
            text = extract_json_from_text(item['output'])
            out = parse_missformed_json(text) 
            all_data.append({"sentence": sentence , "output" :  out  })   
        except Exception as e:   
            print(e)
            pass
            err += 1 
        pass
    
print(len(all_data)) 

#### Recursive traverse the json objects and try to get the label and it's values.
Not all the JSON data is formated the same way due to the LLM nature, but doing this I can capture almost 100¤ of the data

In [ ]:
from tqdm import tqdm   

def gather_data_with_label(data, get_label, item): 
    all_items = []

    def traverse_and_gather(data,item):
        if isinstance(data, dict):
            for key, value in data.items():
                label = get_label(key)
                if label != "" and isinstance(value, list) and len(value) > 1:
                    for list_item in value:
                        if isinstance(list_item, str):
                            all_items.append({"sentence": item['sentence'], "label": label, "output": list_item, "item": item}) 
                traverse_and_gather(value,item)  

        elif isinstance(data, list):
            for item2 in data:
                traverse_and_gather(item2,item)  

    traverse_and_gather(data,item)
    return all_items

all_raw_items = [] 
for item in all_data:
    extracted_data = gather_data_with_label(item['output'], get_label,item) 
    all_raw_items.extend(extracted_data) 
    
print(len(all_raw_items))

#### Clean the data and remove data that might be in the wrong section or is just invalid such as if the LLM outputs "Education: [ "N/A" ]

In [ ]:
import nltk 
from nltk.corpus import wordnet
 
def filter_data(data): 
    invalids =set()
    filtered_data = []
    missing_values = [ "", ", ", ". " , " ,", " ", "not specified"
        "n/a", "none", "na", "blank", "missing", "not applicable", 
        "not available", "undefined", "null", "empty", "not provided", 
        "no value", "unspecified", "unknown", "void", "-", "--", "nil", 
        "n.a.", "n.a", "nil", "not set", "not specified", "not given", 
        "no entry", "not entered", "not included", "not valid", 
        "not applicable", 
    ] 

    for entryR in data:
        entry = entryR['output']
        
        entry_processed = entry.strip().replace("_", " ").lower() 
        if entry_processed in missing_values or len(wordnet.synsets(entry_processed.replace(" ",""))) > 0 or len(entry_processed) < 2 or len(entry_processed.split(" ")) <= 2:
            invalids.add(entryR['output'])  
        elif ' ' in entry_processed or len(wordnet.synsets(entry_processed)) == 0:
            filtered_data.append(entryR) 
        else: 
            invalids.add(entryR['output']) 
            
    return filtered_data,invalids
 
filtered_items , invalids = filter_data(all_raw_items)
print(len(filtered_items))
print(len(invalids))

forbidden_keywords_mapping = {
  "Education": ["degree", "diploma", "academic background", "university"],
  "Qualifications": [  "accreditation", "qualification"],
  "WorkExperience": ["work experience", "employment history", "professional experience", "previous positions", "job history", "career history"],
  "TechnicalSkills": ["programming languages", "software tools", "hardware expertise", "technical competencies", "IT skills"], 
  "Certification": ["certification", "accreditation", "professional designation", "certificate", "license"],
  "HardSkill": ["specific skills", "quantifiable skills", "job-specific skills", "technical abilities", "industry-specific skills"],
  "Benefits": ["achievements", "awards", "contributions", "value", "benefits", "advantages"],
  "CompanyCulture": ["values", "personality traits", "work style", "company culture fit", "team", "team player",  "organizational fit"] 
}

all_items = []
for item in filtered_items:
    found = False 
    if item['label'] in forbidden_keywords_mapping:
        for key, value in forbidden_keywords_mapping.items():  
            for labelVal in value:
                if key != item['label'] and labelVal.lower() in  item['output'].lower() :
                    found = True 
                    break  
    if found == False:
        all_items.append(item)

for data in all_items: 
    if data['label']  == "qualifications":
        data['label']  = "Qualifications"  
                     
print(len(all_items))

### Reranker

In [3]:
import threading
from FlagEmbedding import FlagReranker  ,BGEM3FlagModel
import os 
reranker = FlagReranker("BAAI/bge-reranker-base", use_fp16=True) 

In [ ]:
def split_word_at_capital(word):
    splits = [(i, c) for i, c in enumerate(word[1:], 1) if c.isupper()]
    for index, letter in reversed(splits):
        return word[:index], word[index:]

def check_wordnet(words):
    return all(len(wordnet.synsets(word)) > 0  for word in words)

def find_words_split_at_capital(text):
    words = nltk.word_tokenize(text)
    split_words = [split_word_at_capital(word) for word in words if any(c.isupper() for c in word)]
    split_words = [split for split in split_words if split]
    return [split for split in split_words if all(len(word) >= 3 for word in split) and check_wordnet(split)]

def insert_newlines(text):
    split_words = find_words_split_at_capital(text)
    for split in split_words:
        old_word = ''.join(split)
        new_word = '\n'.join(split)
        text = text.replace(old_word, new_word)
    return text 

def split_text(text): 
    chunks = re.findall(r'(.*?(?:\.|\n|$))', text)
    chunks = [chunk.strip() for chunk in chunks if chunk.strip()]
    return chunks

def chunk_text(text): 
    chunks = split_text(insert_newlines(text))  
    chunks = [chunk for chunk in chunks if len(chunk) <= 256]
    return chunks 
 

In [ ]:
grouped_data = {}
for item in all_items:
    age = item['sentence'] 
    if age == "":
        continue
    if age not in grouped_data:
        grouped_data[age] = []   
    grouped_data[age].append(item)
print(len(grouped_data))
     
reRankerDatas = [] 

for sentence, grouped in grouped_data.items(): 
    group_list = list(grouped)  
    rowData = []
    nonLabels = []
    leftOvers = list(group_list)   
     
    for chunk in chunk_text(sentence):
        spacy_format = { 'text' : chunk, 'annotations' : []}
        if len(chunk) > 3: 
            chunk_data = []
            for data in group_list:
                if data['output'].lower() in chunk.lower(): 
                    chunk_data.append(data) 
                    if data in leftOvers:
                        leftOvers.remove(data) 
                    
            if chunk_data:
                for data in chunk_data:
                    spacy_format = {'text': chunk, 'data': data}
                    rowData.append(spacy_format) 
            else: 
                nonLabels.append(chunk)        
            
    if len(rowData) > 0 and len(nonLabels) > 5:   
        for data in rowData:  
            if len(data['data']['output']) > 256:
                continue
            pairs = []
            for nonLabel in nonLabels:
                pairs.append([data['text'],  nonLabel])
                 
            scores = reranker.compute_score(pairs) 
            sorted_pairs = sorted(zip(scores, pairs))
 
            top_pairs = sorted_pairs[:3]   
            nonSelected = []   
            for score, pair in sorted_pairs[:3]: 
                nonSelected.append(pair[1])  
                
            reRankerDatas.append( { "query": data['data']['label'] , "pos" : [ data['text']], "neg" : nonSelected})
            reRankerDatas.append( { "query": data['data']['label'] , "pos" : [ data['data']['output']], "neg" : nonSelected}) 
            
            validLefts = []
            for leftover in validLefts:
                if leftover['label'] == data['data']['label'] and len(leftover['output']) < 256: 
                    reRankerDatas.append( { "query": data['data']['label'] , "pos" : [ leftover['output'] ], "neg" : nonSelected})    
         
                
            validLefts.append(data['data']['output'])
            reRankerDatas.append( { "query": "Breadtext" , "pos" : nonSelected, "neg" : validLefts }) 
            reRankerDatas.append( { "query": data['data']['label'] , "pos" :validLefts, "neg" : nonSelected})
            
print(len(reRankerDatas))      
         

#### If you have too little data or not distinct enough data:

In [ ]:
import random

oppesiteMap = {
    'WorkExperience' : ['Education','CompanyCulture','Benefits','Breadtext'],
    'Education' : [ 'CompanyCulture', 'WorkExperience','Benefits','Breadtext'],
    'CompanyCulture' : ['Education','WorkExperience','Benefits','Breadtext'],
    'Certification'  : ['Education','CompanyCulture','Benefits','Breadtext'],
    'Product'  : ['Education','CompanyCulture','Benefits','Breadtext'],
    'Service'  : ['Education','CompanyCulture','Benefits','Breadtext'],
    'JobDuties'  : ['Education','CompanyCulture','Benefits','Breadtext'],
    'Upcoming'  : ['Education','CompanyCulture','Benefits','Breadtext']
} 
neg_data = {}
for data in reRankerDatas:
    for key, value in oppesiteMap.items():
        if data['query'] in value:
            if key not in neg_data:
                neg_data[key] = []
            neg_data[key].extend(data['pos'])
            
new_data = []            
for item in reRankerDatas: 
    negs = random.sample(neg_data[item['label']], 3)
    new_data.append({ "query": item['label'] , "pos" : [ item['output']], "neg" : negs}) 

reRankerDatas.extend(new_data) 
print(len(reRankerDatas))

##### Format the data that might help reranker infer the meaning behind the text

In [67]:
import json 
import copy
  
lowercase_category_mapping = {
    'Education': 'education', 
    'Certification': 'certification', 
    'Qualifications': 'qualifications', 
    'WorkExperience': 'work experience', 
    'TechnicalSkills': 'hard skills', 
    'SoftSkill': 'soft skills', 
    'HardSkill': 'hard skills', 
    'Skills': 'hard skills', 
    'Proficiencies': 'qualifications', 
    'Benefits': 'benefits', 
    "Breadtext": 'breadtext',
    'CompanyCulture': 'company culture',  
    "Product": 'product',
    "Service": 'service',
    "Upcoming": 'upcoming offerings',
    "JobDuties": 'job duties',
}
 
copyDatas = copy.copy(reRankerDatas)    
for data in copyDatas:
    if isinstance(data['pos'], str):
        data['pos'] = [data['pos']]
    
    for item in data['pos']:
        item = item.replace("\n", "") 
        
    for item in data['neg']:
        item = item.replace("\n", "")    
        
    if "Example of" not in data['query']:
        data['query'] =  lowercase_category_mapping[data['query']] 
            
        data['query'] = "Example of " + data['query'].lower()
          
with open("reRanker_raw_train_dataset.json", "w") as file:
    for item in copyDatas:
        json.dump(item, file)
        file.write('\n')

#### Remove duplicates

In [ ]:
print(len(copyDatas))
def remove_duplicates(json_list):
    unique_json_objects = []
    seen = set()
    for json_obj in json_list:
        json_str = json.dumps(json_obj, sort_keys=True)
        if json_str not in seen:
            unique_json_objects.append(json_obj)
            seen.add(json_str)
    return unique_json_objects

unique_json_objects = remove_duplicates(copyDatas)
print(len(unique_json_objects))

#### Limit the breadtext so it doesn't overpower the dataset

In [85]:
breadtext_objects = [obj for obj in unique_json_objects if obj.get('query') == 'Example of breadtext']
other_objects = [obj for obj in unique_json_objects if obj.get('query') !='Example of breadtext']

random.shuffle(breadtext_objects)
max_breadtext_objects = breadtext_objects[:15000]

selected_objects = max_breadtext_objects + other_objects

In [ ]:
with open("reRanker_train_dataset_cleaned.json", "w") as file:
    for item in selected_objects:
        json.dump(item, file)
        file.write('\n')

In [ ]:
import json
import matplotlib.pyplot as plt
from collections import Counter
 
queries = [obj['query'] for obj in selected_objects]
 
query_counts = Counter(queries)
 
unique_queries = list(query_counts.keys())
counts = list(query_counts.values())
             
# Creating the bar chart
plt.figure(figsize=(8, 6))
plt.bar(unique_queries, counts, color='skyblue')
plt.xlabel('Query')
plt.ylabel('Count')
plt.title('Distribution of Queries')
plt.xticks(rotation=45)
plt.tight_layout()

# Displaying the bar chart
plt.show()
query_counts = Counter(queries)

# Displaying the results
for query, count in query_counts.items():
    print(f"Query: {query}, Count: {count}")


##### After fine-tuning the reranker model, convert to ONNX if you want to use it in e.g .NET

In [2]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# load model and tokenizer
model_id = r'./Reranker_finetuned'
model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

dummy_model_input = tokenizer([['what is panda?', 'hi']], padding=True, truncation=True, return_tensors='pt', max_length=512)
 
# export
torch.onnx.export(
    model, 
    tuple(dummy_model_input.values()),
    f="torch-model.onnx",  
    input_names=['input_ids', 'attention_mask'], 
    output_names=['logits'], 
    dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence'}, 
                  'attention_mask': {0: 'batch_size', 1: 'sequence'}, 
                  'logits': {0: 'batch_size', 1: 'sequence'}}, 
    
    do_constant_folding=True, 
    opset_version=13, 
)